# **DRiM CHALLENGE 2022**
First tasks form October to November:
- Knowledge of the topic and the available data
- Design by the teams of the prototype of their solution
• Selection by the training managers of the 2 teams 


# TO DO



*   clustering de ligne
    * https://umap-learn.readthedocs.io/en/latest/aligned_umap_politics_demo.html
    * scitkitlearn

*   stumpy python
* centre reduire les donnees en focntion des besoins
* essayer de réduire de dimension les données 8*8D en 8D 
    * peut être open-TSNE mais risque de perdre des données https://opentsne.readthedocs.io/







# 0. Subject and data



- Preliminary analysis:
> - Presentation of the database
> - Stationarity analyses
> - Possible transformations on the variables
> - Analysis of correlations
> - Verification of the hypotheses underlying the models used

- Predictions models:
> - Traditional modelling techniques: testing ARDL, ARIMAX ... -> CI integration
> - Recent modelling techniques: testing relevant models
> - Provide interpretation of results for both families of models

- Performance:
> - Present and interpret the results of the main performance indicators to validate the forecast model (BIC, AIC,...)
> - Validate the quality of the forecasts (Back-testing on the 2nd day): compare the projection with the observation and present/interpret the synthetic indicators (RMSE MAE,...)

- Dynamic restitution:
> - Powerpoint: presentation of the objective, approach, data, results
> - Data visualisation
> - Oral presentation of the content in 15 min

# 1. Libraries and Google Drive

In [1]:
# Import Google Drive
# You have to shortcut Basile's file in our Drive to find it
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd # Used to open and analyse the dataset
import matplotlib.pyplot as plt # Used to create charts
import numpy as np # Used to do data processing
import plotly as plty # Used to create charts
import seaborn as sns # Used to create charts
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller # Used to check stationarity 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from scipy.stats import shapiro
sns.set()
import sys
sys.path.append('/content/gdrive/MyDrive/DRIM challenge/')
import tools as tl

ModuleNotFoundError: ignored

# 2. Data importation and reading

In [ ]:
# Same link for everyone:
data = pd.read_sas("/content/gdrive/MyDrive/DRIM challenge/construct_red.sas7bdat")

In [ ]:
# First view of the data:
data.head(15)

In [ ]:
data.tail()

# Focus on the Second DATASET (CHRONIQUE = CHR8)

In [ ]:
data2 = data[data.CHRONIQUE == b'CHR8']
data2_dr = data2.DR
data2_dr.head()

In [ ]:
data2.shape

In [ ]:
data2.head()

In [ ]:
data2.columns

## Données sous la forme statistique

### Tracé du taux de défaut en fonction de la répartition du reste de données

Pour CHR2, nous allons avoir besoin d'une liste des données que nous allons analyser.

In [ ]:
colonnes = [['mean_1', 'median_1', 'p5_1', 'p10_1', 'p25_1', 'p75_1', 'p90_1', 'p95_1']]

for i in range(2, 9):
  L = list()
  for elt in colonnes[0]:
    L.append(elt[:len(elt)-1]+str(i))
  colonnes.append(L)

Traçons le taux de défaut en fonction de chaque données disponible dans le data set.

In [ ]:
fig, axes = plt.subplots(nrows=8, figsize=(15,45))

List_type_data = ["Note d'octroi recalulée",
                  "Durée écoulée depuis le dernier impayé",
                  "Durée de l'impayé encours",
                  "Durée maximale de l'impayé",
                  "Créance après répartition solde créditeurs",
                  "Ratio nombre d'incidents régularisés enregistrés depuis la date d'entrée en gestion",
                  "Ratio rang du premier impayé régularisé par rapport à la durée prévue du contrat",
                  "Pourcentage apport"]

ax_number=0

for elt in colonnes:
  data2_bis = data2[elt+["DR"]]
  for j in range(len(elt)):
    color = ["c", "r", "b", "y", "k", "g", "b", "m"]
    data2_bis.plot(ax=axes[ax_number],
                    kind="scatter", 
                    grid = True, 
                    x=elt[j], y="DR",
                    color=color[j],
                    legend=True,
                    label=elt[j],
                    title="Taux de défaut en fonction de : " +List_type_data[ax_number])
    
  axes[ax_number].set_xlabel("Répartition des données")

  ax_number += 1

## Analyse de la corrélation

Désormais, nous allons analyser la corrélation entre les données et le taux de défaut :

### Note d'octroi recalulée

In [ ]:
data2_analyse_1 = data2[colonnes[0]+["DR"]]

In [ ]:
correlation_1 = data2_analyse_1.corr()["DR"][:-1].to_frame()
correlation_1 = correlation_1.rename(columns={"DR" : "Correlation avec DR"})
correlation_1 = correlation_1.sort_values(by = "Correlation avec DR", ascending = False)
correlation_1

La corrélation la plus élevée est pour le quantile à 5%. Regardons la valeur des données.

In [ ]:
data2[["p5_1", "DR"]].T

Regardons la valeur moyenne des quantiles à 5%

In [ ]:
print("La moyenne des quantiles à 5% vaut",float(data2[["p5_1"]].mean()))

Le quantile à 5% de la note d'octroi recalculée est assez corrélée avec le taux de défaut.

### Durée écoulée depuis le dernier impayé

In [ ]:
data2_analyse_2 = data2[colonnes[1]+["DR"]]

In [ ]:
correlation_2 = data2_analyse_2.corr()["DR"][:-1].to_frame()
correlation_2 = correlation_2.rename(columns={"DR" : "Correlation avec DR"})
correlation_2 = correlation_2.sort_values(by = "Correlation avec DR", ascending = False)
correlation_2

Analysons les quantiles à 95% de la durée écoulé depuis le dernier impayé

In [ ]:
data2[["p95_2", "DR"]].T

In [ ]:
print("La moyenne des quantiles à 95% vaut",float(data2[["p95_2"]].mean()))

### Durée de l'impayé encours

In [ ]:
data2_analyse_3 = data2[colonnes[2]+["DR"]]
correlation_3 = data2_analyse_3.corr()["DR"][:-1].to_frame()
correlation_3 = correlation_3.rename(columns={"DR" : "Correlation avec DR"})
correlation_3 = correlation_3.sort_values(by = "Correlation avec DR", ascending = False)
correlation_3

In [ ]:
data2[["p75_3", "DR"]].T

In [ ]:
print("La moyenne des quantiles à 75% vaut",float(data2[["p75_3"]].mean()))

Étrange plus la durée de l'impayé en cours augmente plus on penserait que le risque de défaut augmente. Cependant ce n'est pas visiblement le cas.

### Durée maximale de l'impayé

In [ ]:
data2_analyse_4 = data2[colonnes[3]+["DR"]]
correlation_4 = data2_analyse_4.corr()["DR"][:-1].to_frame()
correlation_4 = correlation_4.rename(columns={"DR" : "Correlation avec DR"})
correlation_4 = correlation_4.sort_values(by = "Correlation avec DR", ascending = False)
correlation_4

In [ ]:
data2[["p95_4", "DR"]].T

In [ ]:
print("La moyenne des quantiles à 95% vaut",float(data2[["p95_4"]].mean()))

On remarque que plus la durée maximale de l'impayé augmente, plus la corrélation avec le taux de défaut augmente. Ce à quoi nous devions nous attendre.

### Créance après répartition solde créditeurs

In [ ]:
data2_analyse_5 = data2[colonnes[4]+["DR"]]
correlation_5 = data2_analyse_5.corr()["DR"][:-1].to_frame()
correlation_5 = correlation_5.rename(columns={"DR" : "Correlation avec DR"})
correlation_5 = correlation_5.sort_values(by = "Correlation avec DR", ascending = False)
correlation_5

In [ ]:
data2[["p95_5", "DR"]].T

In [ ]:
print("La moyenne des quantiles à 95% vaut",float(data2[["p95_5"]].mean()))

### Ratio nombre d'incidents régularisés enregistrés depuis la date d'entrée en gestion

In [ ]:
data2_analyse_6 = data2[colonnes[5]+["DR"]]
correlation_6 = data2_analyse_6.corr()["DR"][:-1].to_frame()
correlation_6 = correlation_6.rename(columns={"DR" : "Correlation avec DR"})
correlation_6 = correlation_6.sort_values(by = "Correlation avec DR", ascending = False)
correlation_6

In [ ]:
data2[["p75_6", "DR"]].T

In [ ]:
print("La moyenne des quantiles à 75% vaut",float(data2[["p75_6"]].mean()))

### Ratio rang du premier impayé régularisé par rapport à la durée prévue du contrat

In [ ]:
data2_analyse_7 = data2[colonnes[6]+["DR"]]
correlation_7 = data2_analyse_7.corr()["DR"][:-1].to_frame()
correlation_7 = correlation_7.rename(columns={"DR" : "Correlation avec DR"})
correlation_7 = correlation_7.sort_values(by = "Correlation avec DR", ascending = False)
correlation_7

In [ ]:
data2[["p10_7", "DR"]].T

In [ ]:
print("La moyenne des quantiles à 10% vaut",float(data2[["p10_7"]].mean()))

### Pourcentage apport

In [ ]:
data2_analyse_8 = data2[colonnes[7]+["DR"]]
correlation_8 = data2_analyse_8.corr()["DR"][:-1].to_frame()
correlation_8 = correlation_8.rename(columns={"DR" : "Correlation avec DR"})
correlation_8 = correlation_8.sort_values(by = "Correlation avec DR", ascending = False)
correlation_8

In [ ]:
data2[["p95_8", "DR"]].T

In [ ]:
print("La moyenne des quantiles à 95% vaut",float(data2[["p95_8"]].mean()))

## Fonction analyse des variables suivantes 

In [ ]:
def graph_variables(type_variable, data):

  date2_type_variable = data[type_variable+["TRIMESTRE", "DR"]]
  date2_type_variable = date2_type_variable.set_index(keys="TRIMESTRE")

  fig, axes = plt.subplots(nrows=len(type_variable), figsize=(18,4*len(type_variable)))

  color = ["c", "r", "k"]

  for i in range(len(type_variable)):

    date2_type_variable[[type_variable[i]]].plot(ax=axes[i],
                                            kind = "line",
                                            color=color[i],
                                            grid=True,
                                            title="Évolution de "+ type_variable[i])

    date2_type_variable[['DR']].plot(ax=axes[i],
                                    kind = "line",
                                    color="g",
                                    grid=True)


  plt.legend()
  return plt.plot

## Type de client

Désormais, pour les données suivantes nous allons tracer le taux de défaut en fonction du type de client.

* 1 - Particulier
* 2 - Personne physique


In [ ]:
type_client = ['CD_TY_CLI_RCI_1','CD_TY_CLI_RCI_2']

Récupérons les données liées aux types de clients. Et traçons l'évolution de la proportion de chaque type de client. Par dessus nous pouvons rajouter l'évolution du taux de défaut afin de voir si vous observons un éventuel lien.

In [ ]:
graph_variables(type_client, data2)

Je ne remarque pas de lien évident entre le taux de défaut et le type de client.

Testons la normalités des données avec le test de shapiro.

In [ ]:
for client in type_client:
  print(client, shapiro(data2[[client]]))

En fonction de la valeur de alpha, on rejette ou non la normalité

## Mode d'habitation

Faisons la même chose avec le mode d'habitation

* 1 - locataire et autres
* 2 - propriétaire et manquant

In [ ]:
type_habitation = ['CD_MOD_HABI_1', 'CD_MOD_HABI_2']

On obtient le graph suivant :

In [ ]:
graph_variables(type_habitation, data2)

On semble remarquer une certaine corrélation

In [ ]:
for habitation in type_habitation:
  print(habitation, shapiro(data2[[habitation]]))

## État civil

Désormais, pour l'état civil de la personne.

* 1 - la personnne est célibataire ou autre
* 2 - la personne est mariée ou manquant

In [ ]:
etat_civil = ['CD_ETA_CIV_1', 'CD_ETA_CIV_2']

In [ ]:
graph_variables(etat_civil, data2)

On semble remarquer une certaine corrélation

In [ ]:
for et in etat_civil:
  print(et, shapiro(data2[[et]]))

## Code qualité du véhicule

Désormais, pour le code qualité du véhicule.

* 1 - véhicule d'occasion
* 2 - véhicule neuf et autres et manquant

In [ ]:
type_vehicule = ["CD_QUAL_VEH_1", "CD_QUAL_VEH_2"]

In [ ]:
graph_variables(type_vehicule, data2)

On remarque que lors de l'augmentation de proportion de vente de véhicule neufs. le taux de défaut à augmenté également (légèrement)

In [ ]:
for vehicule in type_vehicule:
  print(vehicule, shapiro(data2[[vehicule]]))

## Profession

Désormais, pour la profession de la personne ayant contracté un crédit.

* 1 - Ouvriers et Autres
* 2 - Employés, Autres personnels de services
* 3 - Retraites, Cadres, Professions libérales, Ingénieurs, Agents de Maitrise, ... et Manquants

In [ ]:
type_profession = ["CD_PROF_1", "CD_PROF_2", "CD_PROF_3"]

In [ ]:
graph_variables(type_profession, data2)

In [ ]:
for profession in type_profession:
  print(profession, shapiro(data2[[profession]]))

## Modélisation avec le modèle ARDL

In [ ]:
endo_variable = "DR"
exo_variable = ["CD_PROF_1", "CD_PROF_2", "CD_PROF_3",
                "CD_QUAL_VEH_1", "CD_QUAL_VEH_2",
                'CD_ETA_CIV_1', 'CD_ETA_CIV_2',
                'CD_MOD_HABI_1', 'CD_MOD_HABI_2',
                'CD_TY_CLI_RCI_1','CD_TY_CLI_RCI_2']

In [ ]:
#Import the required modules for TimeSeries data generation:
import statsmodels.api as sm
#Import the required modules for test statistic calculation:
import statsmodels.stats as sm_stat
#Import the required modules for model estimation:
import statsmodels.tsa as smt
#Import the required modules for optimization:
import scipy.optimize as optimize

#We also need additional data:
import statsmodels.formula.api as smf

# Analysis Features

Suppression des données qui ne seront pas prises en compte dans pour les variables du modèle.

In [ ]:
data2_analysis = data2.drop(['DR','TRIMESTRE','CHRONIQUE','dtf_per_trt'],axis = 1)
data2_analysis.head()

Supprimons les variables avec une variance trop faibles :

In [ ]:
var = VarianceThreshold() 
var.fit_transform(data2_analysis) 
data2_analysis.columns[var.get_support()]

In [ ]:
X_chr8 = data2_analysis[data2_analysis.columns[var.get_support()]]
X_init_chr8 = X_chr8
X_init_chr8.head()

Nous pouvons vérifier si certaines données ont une colinéarité trop élevée :

In [ ]:
correlation_matrix_chr8 = X_chr8.corr()

filter = correlation_matrix_chr8>=0.9

couple = list()

for ind in filter.index:
  for col in filter.columns:
    if filter.loc[ind,col] == True and ind != col and sorted([ind,col]) not in couple:
      # We add a sorted list for each couple that fits the condition
      couple.append(sorted([ind,col]))

print("\n".join(sorted(map(lambda x: ' '.join(x),couple))) if couple != [] else None)

On supprime mean_1 median_1 mean_1 p5_1 p10_1 p25_1 p75_1 p90_1 p95_1 p75_2 mean_2 p95_2 p90_4 mean_4 p95_4 mean_5 median_5 p95_5 mean_6 median_6 p25_6 p75_6 p90_6 mean_7 median_7 p10_7 p25_7 p75_7 p90_7 p95_7


In [ ]:
wrong_col = ["mean_1", "median_1", "mean_1", "p5_1", "p10_1", "p25_1", "p75_1", "p90_1", "p95_1", 
             "p75_2", "mean_2", "p95_2",
             "p90_4", "mean_4", "p95_4",
             "mean_5", "median_5", "p95_5",
             "mean_6", "median_6", "p25_6", "p75_6", "p90_6",
             "mean_7", "median_7", "p10_7", "p25_7", "p75_7", "p90_7", "p95_7"]

X_chr8 = X_chr8.drop(wrong_col,axis = 1)
X_chr8.head()

Nous pouvons conserver uniquement les donnees ayant une corrélation élevée avec le taux de défaut.

In [ ]:
data2_analyse = data2[list(X_chr8.columns)+["DR"]]
correlation = data2_analyse.corr()["DR"][:-1].to_frame()
correlation = correlation.rename(columns={"DR" : "Correlation avec DR"})
correlation = correlation.sort_values(by = "Correlation avec DR", ascending = False)
correlation.shape

In [ ]:
taux_min = 0.40

In [ ]:
Xend = X_chr8[X_chr8.columns[abs(correlation["Correlation avec DR"]) >= taux_min]]

In [ ]:
Xend.head()

Ainsi, nous obtenons finalement les variables suivantes :

In [ ]:
for elt in Xend.columns:
  print(elt)
print("-------")
print("Il y a", len(Xend.columns), "variables pour le modèle")

In [ ]:
Xend.shape

Il y a 18 variables avec 26 lignes.

# Normalisation des données

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xend,1000*data2_dr,test_size=0.40,shuffle=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

scaler_train = MinMaxScaler()
scaler_test = MinMaxScaler()

scaler_train.fit(X_train)
scaler_test.fit(X_test)

X_train = pd.DataFrame(scaler_train.transform(X_train),index = X_train.index,columns = X_train.columns)
X_test = pd.DataFrame(scaler_train.transform(X_test),index = X_test.index,columns = X_test.columns)

# Basic Model

In [ ]:
# Simple model without any gridsearch 


models = ["lin","rid","las","ela","sgd","xgb","svr","knn"]
metrics = ["rmse","mse","mae","r2"]
parameters = {"p_xgb":{"booster":"gblinear"}}

tl.summary_ml(X_train,y_train,X_test,y_test,models,parameters=parameters,metrics = metrics)

In [ ]:
tl.summary_ml(X_train,y_train,X_test,y_test,["xgb"],metrics = metrics)

In [ ]:
parameters = {"alpha":np.linspace(0.1,1,20),"tol":[1e-4,1e-3,1e-5],"solver":["auto","svd","cholesky","lsqr","sparse_cg"]}
p_rid,score = tl.params_grid(X_train,y_train,X_test,y_test,'rid',parameters,metric="neg_root_mean_squared_error")
p = {'p_rid':p_rid}
print(score,p_rid)

In [ ]:
tl.summary_ml(X_train,y_train,X_test,y_test,["rid"],parameters=p,metrics = ["rmse","mse","mae","r2"])

In [ ]:
tl.summary_ml(X_train,y_train,X_test,y_test,["rid"],metrics = ["rmse","mse","mae","r2"])

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
rid = Ridge(**p_rid)
rid = rid.fit(X_train,y_train)
y_pred = rid.predict(X_test)

In [ ]:
fig = plt.figure(figsize = (10,10))
plt.plot(y_pred)
plt.plot(np.array(y_test))

In [ ]:
tl.summary_ml(X_train,y_train,X_test,y_test,["xgb"],parameters={"p_xgb":{'alpha': 0, 'booster': 'gblinear', 'lambda': 0, 'updater': 'shotgun'}},metrics = ["mse"])

In [ ]:
plt.plot(y_test)